<a href="https://colab.research.google.com/github/Roshni-seth/Customer-Churn-Prediction/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
Data=pd.read_csv("/content/Churn_Modelling.csv")

In [40]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [41]:
Data.drop(columns=['RowNumber'], inplace=True)
Data.drop(columns=['CustomerId'], inplace=True)
Data.drop(columns=['Surname'], inplace=True)

In [14]:
Data.isnull().sum()

,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0
HasCrCard,0
IsActiveMember,0
EstimatedSalary,0


In [11]:
Data['Geography'].value_counts()

,count
Geography,
France,5014
Germany,2509
Spain,2477


In [12]:
Data['Gender'].value_counts()

,count
Gender,
Male,5457
Female,4543


In [42]:
X = Data.drop(columns=["Exited"])
y = Data["Exited"]

In [44]:
# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# Apply ColumnTransformer: One-Hot Encoding for categorical columns, keep numerical columns unchanged
column_transformer = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),  # Keep numerical columns as is
        ('cat', OneHotEncoder(drop='first', dtype='uint8'), categorical_cols)  # One-Hot Encode categorical columns
    ]
)

In [46]:
# Apply transformations
X_transformed = column_transformer.fit_transform(X)

In [47]:
# Convert transformed X back to DataFrame
encoded_feature_names = column_transformer.get_feature_names_out()
X_encoded = pd.DataFrame(X_transformed, columns=encoded_feature_names)

In [49]:
# Verify transformations
X_encoded.head(), y[:5]

(   num__CreditScore  num__Age  num__Tenure  num__Balance  num__NumOfProducts  \
 0             619.0      42.0          2.0          0.00                 1.0   
 1             608.0      41.0          1.0      83807.86                 1.0   
 2             502.0      42.0          8.0     159660.80                 3.0   
 3             699.0      39.0          1.0          0.00                 2.0   
 4             850.0      43.0          2.0     125510.82                 1.0   
 
    num__HasCrCard  num__IsActiveMember  num__EstimatedSalary  \
 0             1.0                  1.0             101348.88   
 1             0.0                  1.0             112542.58   
 2             1.0                  0.0             113931.57   
 3             0.0                  0.0              93826.63   
 4             1.0                  1.0              79084.10   
 
    cat__Geography_Germany  cat__Geography_Spain  cat__Gender_Male  
 0                     0.0                   0.0   

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [53]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np


rand_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
rand_forest_model.fit(X_train, y_train)


feature_importances = rand_forest_model.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X_encoded.columns, 'Importance': feature_importances})


feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)


feature_importance_df.head(10)


,Feature,Importance
1,num__Age,0.236922
7,num__EstimatedSalary,0.147558
0,num__CreditScore,0.143338
3,num__Balance,0.141612
4,num__NumOfProducts,0.131486
2,num__Tenure,0.082080
6,num__IsActiveMember,0.040725
8,cat__Geography_Germany,0.026190
5,num__HasCrCard,0.018454
10,cat__Gender_Male,0.018421


In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Train Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

# Predictions
y_pred_log_reg = log_reg.predict(X_test)

# Evaluate model performance
log_reg_acc = accuracy_score(y_test, y_pred_log_reg)
log_reg_report = classification_report(y_test, y_pred_log_reg)

print(log_reg_acc)
print(log_reg_report)

0.816
              precision    recall  f1-score   support

           0       0.83      0.97      0.89      1607
           1       0.59      0.21      0.31       393

    accuracy                           0.82      2000
   macro avg       0.71      0.59      0.60      2000
weighted avg       0.79      0.82      0.78      2000



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
from sklearn.tree import DecisionTreeClassifier

# Train Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42, max_depth=10)  # Limit depth to prevent overfitting
dt_model.fit(X_train, y_train)

# Predictions
y_pred_dt = dt_model.predict(X_test)

# Evaluate model performance
dt_acc = accuracy_score(y_test, y_pred_dt)
dt_report = classification_report(y_test, y_pred_dt)

print(dt_acc)
print(dt_report)

0.8405
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1607
           1       0.62      0.50      0.55       393

    accuracy                           0.84      2000
   macro avg       0.75      0.71      0.73      2000
weighted avg       0.83      0.84      0.83      2000



In [62]:
from sklearn.ensemble import RandomForestClassifier

# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X_train, y_train)

# Predictions
y_pred_rf = rf_model.predict(X_test)

# Evaluate model performance
rf_acc = accuracy_score(y_test, y_pred_rf)
rf_report = classification_report(y_test, y_pred_rf)

print(rf_acc)
print(rf_report)

0.866
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      1607
           1       0.78      0.45      0.57       393

    accuracy                           0.87      2000
   macro avg       0.83      0.71      0.74      2000
weighted avg       0.86      0.87      0.85      2000



In [63]:
from xgboost import XGBClassifier

# Train XGBoost model
xgb_model = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Predictions
y_pred_xgb = xgb_model.predict(X_test)

# Evaluate model performance
xgb_acc = accuracy_score(y_test, y_pred_xgb)
xgb_report = classification_report(y_test, y_pred_xgb)

print(xgb_acc)
print(xgb_report)

0.864
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1607
           1       0.71      0.51      0.60       393

    accuracy                           0.86      2000
   macro avg       0.80      0.73      0.76      2000
weighted avg       0.85      0.86      0.86      2000



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [18:34:56] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
